In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.misc import derivative
import plotly.graph_objects as go
from scipy.optimize import curve_fit

2nd derivates are nice but they won't help find the Cq.
back ground reading:
"Automatically finding starting values for sigmoid curve", https://stats.stackexchange.com/questions/242594/automatically-finding-starting-values-for-a-sigmoid-curve
Calculate Ct/Cq from amplication curves, https://github.com/ewallace/tidyqpcr/issues/19
"How to smooth a curve in the right way?", https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way
"Python baseline correction library", Python baseline correction library

 "The package contains pre-processor functions
 (smoothing, imputation, background range detection, baseline correction and normalization),
 a single-blinded randomized rating function, quality analysis summary functions,
 an amplification efficiency function, an amplification curve simulator
 and a report generation function (Supplementary Section S4)."



The supplement in chipPCR is the best place to begin. Follow its steps. 
Available in Mendeley
https://academic.oup.com/bioinformatics/article/31/17/2900/182797

In [6]:
def logistic5(x, a, b, c, d, g): # spiess et al. 5P logistic curve
    # a is ground (bottom) asymptote
    # b is slope
    # c is inflection point
    # d is maximum (top) asymptote
    # g is "...the additional asymmetry parameter. Setting parameter g = 1 yields the four-parameter curves..."
    numerator = a-d
    denominator = (1+(x/c)**b)**g
    return (d+(numerator/denominator))

# provide initial ground_asymptote based on data
def ground_asymptote (fluor):
    # average first n cycles or so 
    # I think Anitoa does first 13 cycles
    n = 5
    return (sum(fluor[0:n]) / len(fluor[0:n]))

# provide initial ground_asymptote based on data
def max_asymptote (fluor):
    # average last n cycles or so 
    # I think Anitoa does first 13 cycles
    m = 5
    return (sum(fluor[-m:]) / len(fluor[-m:]))

# data for iconic, well-behaved rxn
iconic_y = [-23.72, -47.4, -27.93, -4.51, 0.65, 12.79, 29.96, 50.24, 79.85, 129.94, 203.74, 321.5, 488.96, 729.49, 1047.78, 1479.57, 1963.18, 2499.28, 3025.52, 3514.9, 3919.58, 4241.47, 4475.93, 4648.39, 4768.29, 4840.66, 4892.92, 4930.58, 4962.46, 4970.39, 4989.07, 4989.16, 4997.8, 4995.48, 5002.82, 5001.02, 5003.96, 5008.01, 5005.17, 5004.64, 5004.83]
cycles = np.arange(1,42)

# initial guesses: a, b, c, d, g
# p0 = [-7.498, 9.945, 18, 5013, 0.5451] #graphpad values with this dataset
# p0 = [4.04930663e+00, 6.34869459e+00, 2.12543050e+01, 5.00090309e+03, 2.31685219e+00] #scipy output using graphpad
p0 = (ground_asymptote(iconic_y), 10, 20, max_asymptote(iconic_y), 1) # g = 1 is no asymmetry

# parameters (optimal values), parameters covariance
popt, pcov = curve_fit(logistic5, cycles, iconic_y, p0) #least sq = dogbox
print (popt)
# calc opt fit
opt_y = logistic5(cycles, *popt)

# plot data vs opt fit
fig = go.Figure()
fig.add_trace(go.Scatter(x=cycles, y=iconic_y, mode='lines+markers', name='data', marker=dict(size=4, color='black'), line=dict(color='black', width=0.5)))
fig.add_trace(go.Scatter(x=cycles, y=opt_y, mode='lines+markers', name='fit', marker=dict(size=4, color='red'), line=dict(color='red', width=0.5)))

fig.update_layout(
    autosize=False,
    width=800,
    height=500, 
    title="Fitting qPCR Data vs Optimal Fit (Logistic 5PL)",
    xaxis_title="Cycle",
    yaxis_title="RFU",
    legend_title="Source",)
fig.show()


[4.04917778e+00 6.34869386e+00 2.12543067e+01 5.00090309e+03
 2.31685306e+00]
